<a href="https://colab.research.google.com/github/whoami-Lory271/thesis-project/blob/main/thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and imports


In [1]:
!pip install pytorch-lightning==2.0.1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import logging
from google.colab import drive
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# create logger
log = logging.getLogger('model_application')
log.setLevel(logging.DEBUG)

# # create console handler and set level to debug
# ch = logging.StreamHandler()
# ch.setLevel(logging.INFO)

# # create formatter
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # add formatter to ch
# ch.setFormatter(formatter)

# # add ch to logger
# logger.addHandler(ch)

In [5]:
# 'application' code
log.debug('debug message')
log.info('info message')
# logger.warning('warn message')
# logger.error('error message')
# logger.critical('critical message')

DEBUG:model_application:debug message
INFO:model_application:info message


# Constants

In [6]:
#paths
ELECTRICITY = "electricity"
ROOT_FOLDER = "/content/drive/MyDrive/Università/Magistrale/Tesi/code"

#hyperparameters
BATCH_SIZE = 16

# Preprocessing

## Datasets

In [7]:
datasets_path = {
    ELECTRICITY: ROOT_FOLDER + "/datasets/electricity"
}

datasets_name = {
    ELECTRICITY: "/LD2011_2014.txt"    
}
datasets_processed_name = {
    ELECTRICITY: "/electricity.pkl"
}

### Electricity

**Preprocessing**

In [8]:
# df = pd.read_csv(datasets_path[ELECTRICITY] + datasets_name[ELECTRICITY], sep = ';')
# df.rename(columns={df.columns[0]: 'Date'},inplace=True)
# df.to_pickle(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY])

In [9]:
df = pd.read_pickle(datasets_path[ELECTRICITY] + datasets_processed_name[ELECTRICITY])

In [10]:
class ElectricityDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        ts = self.data.iloc[idx, 1:]
        return ts

class ElectricityDataModule(pl.LightningDataModule):
    def __init__(self, path, batch_size, train_size = 0.6, test_size = 0.4):
        super().__init__()
        self.path = path
        data = pd.read_pickle(path)
        self.train_data, self.validate_data ,self.test_data =  np.split(data, [int(train_size*len(data)), int(test_size*len(data))])     

    # def prepare_data(self):
    #     # download

    def setup(self, stage: str):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.train = ElectricityDataset(self.train_data)
            self.validate = ElectricityDataset(self.validate_data)

        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.test = ElectricityDataset(self.test_data)

        # if stage == "predict":

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, drop_last = True)

    def val_dataloader(self):
        return DataLoader(self.validation, batch_size=self.batch_size, drop_last = True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, drop_last = True)

    # def predict_dataloader(self):
        
